## PROBLEM SET 3

### PART A

implementation of wordgame:  This game is a lot like Scrabble or Text Twist. Letters are dealt to players, who then construct one or more words out of their letters. Each valid word receives a score, based on the length of the word and the letters in that word.

**Dealing**
* A player is dealt a hand of n letters chosen at random (assume n=7 for now).
* he player arranges the hand into as many words as they want out of the letters, but using each letter at most once.
* Some letters may remain unused (these won’t be scored). 

**Scoring**
* The score for the hand is the sum of the score for the words times the length of the word.
* The score for a word is the sum of the points for letters in the word, multiplied by the length of the word, plus 50 points if all n letters are used on the first go.
* Letters are scored as in Scrabble; A is worth 1, B is worth 3, C is worth 3, D is worth 2, E is worth 1, and so on. The dictionary **SCRABBLE_LETTER_VALUES** maps each lowercase letter to its Scrabble letter value.  
* For example, ‘weed’ would be worth 32 points ((4+1+1+2)*4=32), as long as the hand actually has 1 ‘w’, 2 ‘e’s, and 1 ‘d’.
* As another example, if n=7 and you get ‘waybill’ on the first go, it would be worth 155 points ((4+1+4+3+1+1+1)*7=105, +50) for the bonus of using all seven letters). 

In [3]:
import random
import string

VOWELS = 'aeiou'
CONSONANTS = 'bcdfghjklmnpqrstvwxyz'
HAND_SIZE =  7

SCRABBLE_LETTER_VALUES = {
    'a': 1, 'b': 3, 'c': 3, 'd': 2, 'e': 1, 'f': 4, 'g': 2, 'h': 4, 'i': 1, 'j': 8, 'k': 5, 'l': 1, 'm': 3, 'n': 1, 'o': 1, 'p': 3, 'q': 10, 'r': 1, 's': 1, 't': 1, 'u': 1, 'v': 4, 'w': 4, 'x': 8, 'y': 4, 'z': 10
}

WORDLIST_FILENAME = "words_3.txt"

def load_words():
    """
    Returns a list of valid words. Words are strings of lowercase letters.
    
    Depending on the size of the word list, this function may
    take a while to finish.
    """
    print ("Loading word list from file...")
    # inFile: file
    inFile = open(WORDLIST_FILENAME, 'r')
    # wordlist: list of strings
    wordlist = []
    for line in inFile:
        wordlist.append(line.strip().lower())
    print ("  ", len(wordlist), "words loaded.")
    return wordlist

def get_frequency_dict(sequence):
    """
    Returns a dictionary where the keys are elements of the sequence
    and the values are integer counts, for the number of times that
    an element is repeated in the sequence.

    sequence: string or list
    return: dictionary
    """
    # freqs: dictionary (element_type -> int)
    freq = {}
    for x in sequence:
        freq[x] = freq.get(x,0) + 1
    return freq

def get_word_score(word, n):
    """
    Returns the score for a word. Assumes the word is a
    valid word.

    The score for a word is the sum of the points for letters
    in the word multiplied by the length of the word, plus 50
    points if all n letters are used on the first go.

    Letters are scored as in Scrabble; A is worth 1, B is
    worth 3, C is worth 3, D is worth 2, E is worth 1, and so on.

    word: string (lowercase letters)
    returns: int >= 0
    """
    word = get_frequency_dict(word)
    score = 0
    for letter in word:
         if letter in SCRABBLE_LETTER_VALUES:
            # increase score by letter value multiply by the number of time the letter is in the word.
            score += SCRABBLE_LETTER_VALUES[letter]*word[letter]
    score*=sum(word.values())
    #check is word equal to number of letter in hand
    if sum(word.values()) == n: 
        score+=50
    return score
    
def display_hand(hand):
    """
    Displays the letters currently in the hand.

    For example:
       display_hand({'a':1, 'x':2, 'l':3, 'e':1})
    Should print out something like:
       a x x l l l e
    The order of the letters is unimportant.

    hand: dictionary (string -> int)
    """
    for letter in hand.keys():
        for j in range(hand[letter]):
             print (letter, end=' ')      # print all on the same line
    print()                               # print an empty line
    

def deal_hand(n):
    """
    Returns a random hand containing n lowercase letters.
    At least n/3 the letters in the hand should be VOWELS.

    Hands are represented as dictionaries. The keys are
    letters and the values are the number of times the
    particular letter is repeated in that hand.

    n: int >= 0
    returns: dictionary (string -> int)
    """
    hand={}
    num_vowels = int(n / 3)
    
    for i in range(num_vowels):
        x = VOWELS[random.randrange(0,len(VOWELS))]
        hand[x] = hand.get(x, 0) + 1
        
    for i in range(num_vowels, n):    
        x = CONSONANTS[random.randrange(0,len(CONSONANTS))]
        hand[x] = hand.get(x, 0) + 1
     
    return hand

def update_hand(hand, word):
    """
    Assumes that 'hand' has all the letters in word.
    In other words, this assumes that however many times
    a letter appears in 'word', 'hand' has at least as
    many of that letter in it. 

    Updates the hand: uses up the letters in the given word
    and returns the new hand, without those letters in it.

    Has no side effects: does not modify hand.

    word: string
    hand: dictionary (string -> int)    
    returns: dictionary (string -> int)
    """   
    temp_hand = hand.copy()
    for letter in word:
            temp_hand[letter]-=1 
    return temp_hand

def is_valid_word(word, hand, word_list):
    """
    Returns True if word is in the word_list and is entirely
    composed of letters in the hand. Otherwise, returns False.
    Does not mutate hand or word_list.
    
    word: string
    hand: dictionary (string -> int)
    word_list: list of lowercase strings
    """
    valid_word = False
    temp_hand = hand.copy()
    if word in word_list: #check is a word is in wordlist
        for letter in word:  #check for each letter if is in hand and if we have multiples times that letter
            if letter in temp_hand and temp_hand[letter]>=1:
                temp_hand[letter]-=1
                valid_word = True           
            else:  #if fail once, output from loop
                valid_word = False  
                break
    return valid_word
    
def calculate_handlen(hand):
    """ 
    Returns the length (number of letters) in the current hand.
    
    hand: dictionary (string int)
    returns: integer
    """
    handlen = 0
    for v in hand.values():
        handlen += v
    return handlen

def play_hand(hand, word_list):

    """
    Allows the user to play the given hand, as follows:

    * The hand is displayed.
    
    * The user may input a word.

    * An invalid word is rejected, and a message is displayed asking
      the user to choose another word.

    * When a valid word is entered, it uses up letters from the hand.

    * After every valid word: the score for that word is displayed,
      the remaining letters in the hand are displayed, and the user
      is asked to input another word.

    * The sum of the word scores is displayed when the hand finishes.

    * The hand finishes when there are no more unused letters.
      The user can also finish playing the hand by inputing a single
      period (the string '.') instead of a word.

      hand: dictionary (string -> int)
      word_list: list of lowercase strings
      
    """
    score = 0
    n=sum(hand.values())
    
    while sum(hand.values()) > 0:
        print('Current hand:',end = " ") 
        display_hand(hand)
        word = input('Enter word, or a "." to indicate that you are finished:')
        if word == '.':
                print('Total score: %d points.' %score)
                return 
        else:
            isValid=is_valid_word(word, hand, word_list)
            if not isValid:
                print ('That is not a valid word. Please choose another word')           
            else: # if valid
                point = get_word_score(word, n) # calculate points
                score += point # add points to total score
                print ('"%s" earned %d points. Total: %d points' % (word, point, score))# display points and total
                hand = update_hand(hand, word) # update hand
    print ('Run out of letters. Total score: %d points.' % score)
                
def play_game(word_list):
    """
    Allow the user to play an arbitrary number of hands.

    * Asks the user to input 'n' or 'r' or 'e'.

    * If the user inputs 'n', let the user play a new (random) hand.
      When done playing the hand, ask the 'n' or 'e' question again.

    * If the user inputs 'r', let the user play the last hand again.

    * If the user inputs 'e', exit the game.

    * If the user inputs anything else, ask them again.
    """
    hand = deal_hand(HAND_SIZE) # random init define at start of the code
    while True:
        cmd = input('Enter n to deal a new hand, r to replay the last hand, or e to end game: ')
        if cmd == 'n':
            hand = deal_hand(HAND_SIZE)
            play_hand(hand.copy(), word_list)
            print ()
        elif cmd == 'r':
            play_hand(hand.copy(), word_list)
            print ()
        elif cmd == 'e':
            break
        else:
            print ("Invalid command.")
    

# Build data structures used for entire session and play game
if __name__ == '__main__':
    word_list = load_words()
    play_game(word_list)

Loading word list from file...
   83667 words loaded.


Enter n to deal a new hand, r to replay the last hand, or e to end game:  e


In [4]:
#
# Test code
#
def test_deal_hand():
    """
    Unit test for deal_hand.
    """
    
    # (A)
    # Basic test, see if the right kind of dictionary is
    # being returned.
    hand = deal_hand(HAND_SIZE)
    if not type(hand) is dict:
        print ("FAILURE: test_deal_hand()")
        print ("\tUnexpected return type:", type(hand))
        
        return # exit function

    num = 0
    for k in hand.keys():
        if (not type(k) is str) or (not type(hand[k]) is int):
            print ("FAILURE: test_deal_hand()")
            print ("\tUnexpected type of dictionary: string -> int expected, but was", type(k), "->", type(hand[k]))

            return # exit function
        elif not k in "abcdefghijklmnopqrstuvwxyz":
            print ("FAILURE: test_deal_hand()")
            print ("\tDictionary keys are not lowercase letters.")

            return # exit function
        else:
            num += hand[k]
            
    if num != HAND_SIZE:
            print ("FAILURE: test_deal_hand()")
            print ("\tdeal_hand() returned more letters than it was asked to.")
            print ("\tAsked for a hand of size", HAND_SIZE, "but it returned a hand of size", num)

            return # exit function
        
    # (B)
    # Tests randomness..
    repeats=0
    hand1 = deal_hand(HAND_SIZE)
    for i in range(20):                
        hand2 = deal_hand(HAND_SIZE)
        if hand1 == hand2:
            repeats += 1
        hand1 = hand2
        
    if repeats > 10:
        print ("FAILURE: test_deal_hand()")
        print ("\tSame hand returned", repeats, "times by deal_hand(). This is HIGHLY unlikely.")
        print ("\tIs the deal_hand implementation really using random numbers?")

        return # exit function
    
    print ("SUCCESS: test_deal_hand()")

def test_get_word_score():
    """
    Unit test for get_word_score
    """
    failure=False
    # dictionary of words and scores
    words = {("", 7):0, ("it", 7):4, ("was", 7):18, ("scored", 7):54, ("waybill", 7):155, ("outgnaw", 7):127, ("outgnawn", 8):146}
    for (word, n) in words.keys():
        score = get_word_score(word, n)
        if score != words[(word, n)]:
            print ("FAILURE: test_get_word_score()")
            print ("\tExpected", words[(word, n)], "points but got '" + str(score) + "' for word '" + word + "', n=" + str(n))
            failure=True
    if not failure:
        print ("SUCCESS: test_get_word_score()")

def test_update_hand():
    """
    Unit test for update_hand
    """
    # test 1
    hand = {'a':1, 'q':1, 'l':2, 'm':1, 'u':1, 'i':1}
    word = "quail"

    hand2 = update_hand(hand.copy(), word)
    expected_hand1 = {'l':1, 'm':1}
    expected_hand2 = {'a':0, 'q':0, 'l':1, 'm':1, 'u':0, 'i':0}
    if hand2 != expected_hand1 and hand2 != expected_hand2:
        print ("FAILURE: test_update_hand('"+ word +"', " + str(hand) + ")")
        print ("\tReturned: ", hand2, "-- but expected:", expected_hand1, "or", expected_hand2)

        return # exit function
        
    # test 2
    hand = {'e':1, 'v':2, 'n':1, 'i':1, 'l':2}
    word = "evil"

    hand2 = update_hand(hand.copy(), word)
    expected_hand1 = {'v':1, 'n':1, 'l':1}
    expected_hand2 = {'e':0, 'v':1, 'n':1, 'i':0, 'l':1}
    if hand2 != expected_hand1 and hand2 != expected_hand2:
        print ("FAILURE: test_update_hand('"+ word +"', " + str(hand) + ")")    
        print ("\tReturned: ", hand2, "-- but expected:", expected_hand1, "or", expected_hand2)

        return # exit function

    # test 3
    hand = {'h': 1, 'e': 1, 'l': 2, 'o': 1}
    word = "hello"

    hand2 = update_hand(hand.copy(), word)
    expected_hand1 = {}
    expected_hand2 = {'h': 0, 'e': 0, 'l': 0, 'o': 0}
    if hand2 != expected_hand1 and hand2 != expected_hand2:
        print ("FAILURE: test_update_hand('"+ word +"', " + str(hand) + ")")             
        print ("\tReturned: ", hand2, "-- but expected:", expected_hand1, "or", expected_hand2)
        
        return # exit function

    print ("SUCCESS: test_update_hand()")

def test_is_valid_word(word_list):
    """
    Unit test for is_valid_word
    """
    failure=False
    # test 1
    word = "hello"
    hand = get_frequency_dict(word)

    if not is_valid_word(word, hand, word_list):
        print ("FAILURE: test_is_valid_word()")
        print ("\tExpected True, but got False for word: '" + word + "' and hand:", hand)

        failure = True

    # test 2
    hand = {'r': 1, 'a': 3, 'p': 2, 'e': 1, 't': 1, 'u':1}
    word = "rapture"

    if  is_valid_word(word, hand, word_list):
        print ("FAILURE: test_is_valid_word()")
        print ("\tExpected False, but got True for word: '" + word + "' and hand:", hand)

        failure = True        

    # test 3
    hand = {'n': 1, 'h': 1, 'o': 1, 'y': 1, 'd':1, 'w':1, 'e': 2}
    word = "honey"

    if  not is_valid_word(word, hand, word_list):
        print ("FAILURE: test_is_valid_word()")
        print ("\tExpected True, but got False for word: '"+ word +"' and hand:", hand)

        failure = True                        

    # test 4
    hand = {'r': 1, 'a': 3, 'p': 2, 't': 1, 'u':2}
    word = "honey"

    if  is_valid_word(word, hand, word_list):
        print ("FAILURE: test_is_valid_word()")
        print ("\tExpected False, but got True for word: '" + word + "' and hand:", hand)
        
        failure = True

    # test 5
    hand = {'e':1, 'v':2, 'n':1, 'i':1, 'l':2}
    word = "evil"
    
    if  not is_valid_word(word, hand, word_list):
        print ("FAILURE: test_is_valid_word()")
        print ("\tExpected True, but got False for word: '" + word + "' and hand:", hand)
        
        failure = True
        
    # test 6
    word = "even"

    if  is_valid_word(word, hand, word_list):
        print ("FAILURE: test_is_valid_word()")
        print ("\tExpected False, but got True for word: '" + word + "' and hand:", hand)
        print ("\t(If this is the only failure, make sure is_valid_word() isn't mutating its inputs)")        
        
        failure = True        

    if not failure:
        print ("SUCCESS: test_is_valid_word()")


word_list = load_words()
print ("----------------------------------------------------------------------")
print ("Testing get_word_score...")
test_get_word_score()
print ("----------------------------------------------------------------------")
print ("Testing update_hand...")
test_update_hand()
print ("----------------------------------------------------------------------")
print ("Testing is_valid_word...")
test_is_valid_word(word_list)
print ("----------------------------------------------------------------------")
print ("All done!")

Loading word list from file...
   83667 words loaded.
----------------------------------------------------------------------
Testing get_word_score...
SUCCESS: test_get_word_score()
----------------------------------------------------------------------
Testing update_hand...
SUCCESS: test_update_hand()
----------------------------------------------------------------------
Testing is_valid_word...
SUCCESS: test_is_valid_word()
----------------------------------------------------------------------
All done!


### PART B

Teach your computer (SkyNet) to play the game you just built so that you can prove once and for all that computers are inferior to human intellect!

In [5]:
def xcombinations(items, n):
    if n==0: yield []
    else:
        for i in range(len(items)):
            for cc in xcombinations(items[:i]+items[i+1:],n-1):
                yield [items[i]]+cc

def xuniqueCombinations(items, n):
    if n==0: yield []
    else:
        for i in range(len(items)):
            for cc in xuniqueCombinations(items[i+1:],n-1):
                yield [items[i]]+cc
            
def xselections(items, n):
    if n==0: yield []
    else:
        for i in range(len(items)):
            for ss in xselections(items, n-1):
                yield [items[i]]+ss

def xpermutations(items):
    return xcombinations(items, len(items))

def get_perms(hand, n):
    handlist = []
    for key in hand:
        for i in range(hand[key]):
            handlist.append(key)
        l = [] 
        toret = []
        for c in xuniqueCombinations(handlist,n):
            l.append(c)
        for j in l:
            for p in xpermutations(j):
                toret.append("".join(p))
    return toret

if __name__=="__main__":
    print ("Permutations of 'love'")
    for p in xpermutations(['l','o','v','e']): print (''.join(p))

    print ()
    print ("Combinations of 2 letters from 'love'")
    for c in xcombinations(['l','o','v','e'],2): print (''.join(c))

    print ()
    print ("Unique Combinations of 2 letters from 'love'")
    for uc in xuniqueCombinations(['l','o','v','e'],2): print (''.join(uc))

    print ()
    print ("Selections of 2 letters from 'love'")
    for s in xselections(['l','o','v','e'],2): print (''.join(s))

    print ()
    print (map(''.join, list(xpermutations('done'))))

    print ("testing stuff for 6.00")
    print (len(get_perms({"a":2, "b":3, "c":1}, 3)))


Permutations of 'love'
love
loev
lvoe
lveo
leov
levo
olve
olev
ovle
ovel
oelv
oevl
vloe
vleo
vole
voel
velo
veol
elov
elvo
eolv
eovl
evlo
evol

Combinations of 2 letters from 'love'
lo
lv
le
ol
ov
oe
vl
vo
ve
el
eo
ev

Unique Combinations of 2 letters from 'love'
lo
lv
le
ov
oe
ve

Selections of 2 letters from 'love'
ll
lo
lv
le
ol
oo
ov
oe
vl
vo
vv
ve
el
eo
ev
ee

testing stuff for 6.00
120


In [6]:
import time

def comp_choose_word(hand, word_list):
    """
    Given a hand and a word_dict, find the word that gives the maximum value score, and return it.
    This word should be calculated by considering all possible permutations of lengths 1 to HAND_SIZE.

    hand: dictionary (string -> int)
    word_list: list (string)
    """
    n= sum(hand.values())
    permutations = []
    word_scores = {}
    
    for x in range(n): #consider all possible permutions for a hand
        permutations.extend(get_perms(hand, x+1))
        
    for word in permutations: # Calculate score for valid words and save the word and his score in a dictionarie
        if word in word_list:
            score = get_word_score(word, n)
            word_scores[word] = score
            
    if len(word_scores) == 0: #check if there is a valid word for the hand
        max_key = None
    else:
        max_key = max(word_scores, key=word_scores.get)# get the key with max value from the dicitonarie
        
    return max_key
    
def comp_play_hand(hand, word_list):
    """
     Allows the computer to play the given hand, as follows:

     * The hand is displayed.

     * The computer chooses a word using comp_choose_words(hand, word_dict).

     * After every valid word: the score for that word is displayed, 
       the remaining letters in the hand are displayed, and the computer 
       chooses another word.

     * The sum of the word scores is displayed when the hand finishes.

     * The hand finishes when the computer has exhausted its possible choices (i.e. comp_play_hand returns None).

     hand: dictionary (string -> int)
     word_list: list (string)
    """
    score = 0
    n=sum(hand.values())
    
    while sum(hand.values()) > 0:
        print('Current hand:',end = " ") 
        display_hand(hand)
        word = comp_choose_word(hand, word_list)
        if word == None:
            break
        else:
                point = get_word_score(word, n) # calculate points
                score += point # add points to total score
                print ('"%s" earned %d points. Total: %d points' % (word, point, score))# display points and total
                hand = update_hand(hand, word) # update hand
    print ('No more valid words. Total score: %d points.' % score)
                
def play_game(word_list):
    """Allow the user to play an arbitrary number of hands.

    1) Asks the user to input 'n' or 'r' or 'e'.
    * If the user inputs 'n', play a new (random) hand.
    * If the user inputs 'r', play the last hand again.
    * If the user inputs 'e', exit the game.
    * If the user inputs anything else, ask them again.

    2) Ask the user to input a 'u' or a 'c'.
    * If the user inputs 'u', let the user play the game as before using play_hand.
    * If the user inputs 'c', let the computer play the game using comp_play_hand (created above).
    * If the user inputs anything else, ask them again.

    3) After the computer or user has played the hand, repeat from step 1

    word_list: list (string)
    """
    hand = deal_hand(HAND_SIZE) # random init define at start of the code
    while True:
        cmd = input('Enter n to deal a new hand, r to replay the last hand, or e to end game: ')
        while cmd != 'n' and cmd != 'r' and cmd != 'e':
            print ("Invalid command.")
            cmd = input('Enter n to deal a new hand, r to replay the last hand, or e to end game: ')
        
        if cmd == 'e':
            break 
 
        player = input('Enter u to have yourself play, c to have the computer play: ')
        while player != 'u' and player != 'c':
            print ("Invalid command.")
            player = input('Enter u to have yourself play, c to have the computer play: ')

        if cmd == 'n':
            hand = deal_hand(HAND_SIZE)
            if player == 'u':
                play_hand(hand.copy(), word_list)
            else:
                comp_play_hand(hand.copy(), word_list)
        if cmd == 'r':
            if player == 'u':
                play_hand(hand.copy(), word_list)
            else:
                comp_play_hand(hand.copy(), word_list)
#
# Build data structures used for entire session and play game
#
if __name__ == '__main__':
    word_list = load_words()
    play_game(word_list)
    print ("Goodbye!")

Loading word list from file...
   83667 words loaded.


Enter n to deal a new hand, r to replay the last hand, or e to end game:  n
Enter u to have yourself play, c to have the computer play:  c


Current hand: e o g b p h s 
"hopes" earned 50 points. Total: 50 points
Current hand: g b 
No more valid words. Total score: 50 points.


Enter n to deal a new hand, r to replay the last hand, or e to end game:  r
Enter u to have yourself play, c to have the computer play:  c


Current hand: e o g b p h s 
"hopes" earned 50 points. Total: 50 points
Current hand: g b 
No more valid words. Total score: 50 points.


Enter n to deal a new hand, r to replay the last hand, or e to end game:  e


Goodbye!
